In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import torch
from torchvision import transforms
from torch.nn import functional as F
from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode
from xdecoder.BaseModel import BaseModel
from xdecoder import build_model
from utils.visualizer import Visualizer
from utils.distributed import init_distributed

import warnings
warnings.filterwarnings(action='ignore')
logger = logging.getLogger(__name__)

from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from datasets import build_evaluator, build_eval_dataloader
from xdecoder.utils import get_class_names

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 keyWARNING: Cannot find VLPreDataset. Make sure datasets are accessible if you want to use them for training or evaluation.


In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/segvlp_focalt_lang.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
pretrained_pth = '../checkpoints/xdecoder_focalt_best_openseg.pt'

output_root = './output'
image_pth = '../images/animals.png'
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


## Model

In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()

*UNLOADED* sem_seg_head.predictor.pos_embed_caping.weight, Model Shape: torch.Size([77, 512])
$UNUSED$ sem_seg_head.predictor.query_feat_caping.weight, Ckpt Shape: torch.Size([77, 512])


In [6]:
t = []
t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
transform = transforms.Compose(t)

thing_classes = ['zebra','antelope','giraffe','ostrich','sky','water','grass','sand','tree']
thing_colors = [random_color(rgb=True, maximum=255).astype(np.int64).tolist() for _ in range(len(thing_classes))]
thing_dataset_id_to_contiguous_id = {x:x for x in range(len(thing_classes))}

MetadataCatalog.get("demo").set(
    thing_colors=thing_colors,
    thing_classes=thing_classes,
    thing_dataset_id_to_contiguous_id=thing_dataset_id_to_contiguous_id,
)

model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(thing_classes + ["background"], is_eval=False)
metadata = MetadataCatalog.get('demo')
model.model.metadata = metadata
model.model.sem_seg_head.num_classes = len(thing_classes)
model.eval()

BaseModel(
  (model): GeneralizedXdecoder(
    (backbone): D2FocalNet(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): FocalModulationBlock(
              (dw1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (modulation): FocalModulation(
                (f): Linear(in_features=96, out_features=196, bias=True)
                (h): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
                (act): GELU(approximate='none')
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (foca

In [7]:
num_layers = opt['MODEL']['BACKBONE']['FOCAL']['DEPTHS']
out_indices = opt['MODEL']['BACKBONE']['FOCAL']['OUT_INDICES']
focal_levels = opt['MODEL']['BACKBONE']['FOCAL']['FOCAL_LEVELS']
num_layers, out_indices, focal_levels

([2, 2, 6, 2], [0, 1, 2, 3], [3, 3, 3, 3])

### Backbone

In [8]:
pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

with torch.no_grad():
    image_ori = Image.open(image_pth).convert('RGB')
    width = image_ori.size[0]
    height = image_ori.size[1]
    image = transform(image_ori)
    image = np.asarray(image)
    image_ori = np.asarray(image_ori)
    images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
    batch_inputs = [{'image': images, 'height': height, 'width': width}]
    # outputs = model.forward(batch_inputs)

    images = [x["image"].to("cuda") for x in batch_inputs]
    images = [(x - pixel_mean) / pixel_std for x in images]
    
    images = ImageList.from_tensors(images, 32)
    print(f"Image shape: {images.tensor.shape}")

    ######################################################
    ## Backbone
    # output = model.model.backbone(images.tensor)
    # print(output.shape)
    ######################################################
    ## Backbone Inner Code

    # 1. Patch Embedding
    patch_embed = deepcopy(model.model.backbone.patch_embed)
    x = patch_embed(images.tensor)
    print(f"patch_embed shape: {x.shape}")
    Wh, Ww = x.size(2), x.size(3)
    x = x.flatten(2).transpose(1, 2)
    print(f"flatten shape: {x.shape}")

    # 2. Dropout
    pos_drop = deepcopy(model.model.backbone.pos_drop)
    x = pos_drop(x)

    # 3. Layers
    outs = {}
    for i in range(len(num_layers)):
        layer = model.model.backbone.layers[i]
        x_out, H, W, x, Wh, Ww = layer(x, Wh, Ww)
        if i in out_indices:
            norm_layer = getattr(model.model.backbone, f'norm{i}')
            x_out = norm_layer(x_out)

            out = x_out.view(-1, H, W, model.model.backbone.num_features[i]).permute(0, 3, 1, 2).contiguous()
            outs["res{}".format(i + 2)] = out
            
    if len(model.model.backbone.out_indices) == 0:
        outs["res5"] = x_out.view(-1, H, W, model.model.backbone.num_features[i]).permute(0, 3, 1, 2).contiguous()
    
    print("\nBackbone output")
    outputs = {}
    for k in outs.keys():
        if k in model.model.backbone._out_features:
            outputs[k] = outs[k]
            print(f"{k}: {outs[k].shape}")

Image shape: torch.Size([1, 3, 800, 1344])
patch_embed shape: torch.Size([1, 96, 200, 336])
flatten shape: torch.Size([1, 67200, 96])

Backbone output
res2: torch.Size([1, 96, 200, 336])
res3: torch.Size([1, 192, 100, 168])
res4: torch.Size([1, 384, 50, 84])
res5: torch.Size([1, 768, 25, 42])


## SEMANTIC SEGMENTATION HEAD (xdecoder_head.py)

### PIXEL DECODER (transformer_encoder_fpn.py)

In [9]:
## XDecoder Head
## 1. Pixel Decoder

with torch.no_grad():
    ## Pixel Decoder Inner Code
    multi_scale_features = []
    num_cur_levels = 0
    
    # Reverse feature maps into top-down order (from low to high resolution)
    for idx, f in enumerate(model.model.sem_seg_head.pixel_decoder.in_features[::-1]):
        x = outputs[f]
        lateral_conv = model.model.sem_seg_head.pixel_decoder.lateral_convs[idx]
        output_conv = model.model.sem_seg_head.pixel_decoder.output_convs[idx]
        if lateral_conv is None:
            print(f"x: {x.shape}")
            transformer = model.model.sem_seg_head.pixel_decoder.input_proj(x)
            print(f"transformer: {transformer.shape}")
            pos = model.model.sem_seg_head.pixel_decoder.pe_layer(x)
            # print(transformer.shape)
            transformer = model.model.sem_seg_head.pixel_decoder.transformer(transformer, None, pos)
            y = output_conv(transformer)
            print(f"transformer conv: {y.shape}")
            # save intermediate feature as input to Transformer decoder
            print(f"transformer: {transformer.shape}")
            transformer_encoder_features = transformer
        else:
            print(f"x: {x.shape}")
            cur_fpn = lateral_conv(x)
            print(f"cur_fpn: {cur_fpn.shape}")
            # Following FPN implementation, we use nearest upsampling here
            print(f"y: {y.shape}")
            y = cur_fpn + F.interpolate(y, size=cur_fpn.shape[-2:], mode="nearest")
            print(f"y: {y.shape}")
            y = output_conv(y)
            print(f"ouput_y: {y.shape}")
            # print()
        if num_cur_levels < model.model.sem_seg_head.pixel_decoder.maskformer_num_feature_levels:
            multi_scale_features.append(y)
            num_cur_levels += 1

    mask_features = model.model.sem_seg_head.pixel_decoder.mask_features(y) if model.model.sem_seg_head.pixel_decoder.mask_on else None
    
    print("\nPixel Decoder output")
    print(f"mask features: {mask_features.shape}")
    print(f"transformer enoder features: {transformer_encoder_features.shape}\n")
    for multi_scale_feature in multi_scale_features:
        print(f"multi_scale_feature: {multi_scale_feature.shape}")

x: torch.Size([1, 768, 25, 42])
transformer: torch.Size([1, 512, 25, 42])
transformer conv: torch.Size([1, 512, 25, 42])
transformer: torch.Size([1, 512, 25, 42])
x: torch.Size([1, 384, 50, 84])
cur_fpn: torch.Size([1, 512, 50, 84])
y: torch.Size([1, 512, 25, 42])
y: torch.Size([1, 512, 50, 84])
ouput_y: torch.Size([1, 512, 50, 84])
x: torch.Size([1, 192, 100, 168])
cur_fpn: torch.Size([1, 512, 100, 168])
y: torch.Size([1, 512, 50, 84])
y: torch.Size([1, 512, 100, 168])
ouput_y: torch.Size([1, 512, 100, 168])
x: torch.Size([1, 96, 200, 336])
cur_fpn: torch.Size([1, 512, 200, 336])
y: torch.Size([1, 512, 100, 168])
y: torch.Size([1, 512, 200, 336])
ouput_y: torch.Size([1, 512, 200, 336])

Pixel Decoder output
mask features: torch.Size([1, 512, 200, 336])
transformer enoder features: torch.Size([1, 512, 25, 42])

multi_scale_feature: torch.Size([1, 512, 25, 42])
multi_scale_feature: torch.Size([1, 512, 50, 84])
multi_scale_feature: torch.Size([1, 512, 100, 168])


### PREDICTOR (xdecoder.py)

In [10]:
predictor = model.model.sem_seg_head.predictor

In [15]:
##############################################################################################################################
###### XDecoder Head ######
###### 2. Predictor ######
num_feature_levels = 3
with torch.no_grad():
    # predictions = model.model.sem_seg_head.predictor(multi_scale_features, mask_features, mask=None, target_queries=None, target_vlp=None, task='seg', extra={})
    # print("\npredictor output")
    # print(predictions.keys())
    x = multi_scale_features
    src = []
    pos = []
    size_list = []
    for i in range(num_feature_levels):
        size_list.append(x[i].shape[-2:])
        pos.append(predictor.pe_layer(x[i], None).flatten(2))
        # print(predictor.level_embed.weight[i][None, :, None].shape)
        # print(predictor.level_embed.weight[i])
        src.append(predictor.input_proj[i](x[i]).flatten(2) + predictor.level_embed.weight[i][None, :, None])

        # flatten NxCxHxW to HWxNxC
        pos[-1] = pos[-1].permute(2, 0, 1)
        src[-1] = src[-1].permute(2, 0, 1)
    
        print(f"{i}: {x[i].shape} -> {pos[-1].shape}")
        print(f"{i}: {x[i].shape} -> {pos[-1].shape}")

    _, bs, _ = src[0].shape

    query_embed = predictor.query_embed.weight.unsqueeze(1).repeat(1, bs, 1)
    output = predictor.query_feat.weight.unsqueeze(1).repeat(1, bs, 1)

    print(f"query_pos: {query_embed.shape}")
    print(f"query_feature: {output.shape}")

    predictions_class = []
    predictions_mask = []
    predictions_bbox = []
    predictions_caption = []
    predictions_captioning = []

    # refer forward_prediction_heads func
    # seg
    self_tgt_mask = predictor.self_attn_mask[:,:predictor.num_queries,:predictor.num_queries].repeat(output.shape[1]*predictor.num_heads, 1, 1)
    print(f"self_tgt_mask: {predictor.self_attn_mask.shape}\n")
    
    decoder_output = predictor.decoder_norm(output)
    decoder_output = decoder_output.transpose(0, 1)
    print(f"decoder_output: {decoder_output.shape}")

    norm_decoder_output = decoder_output / (decoder_output.norm(dim=-1, keepdim=True) + 1e-7)
    print(f"norm decoder_output: {norm_decoder_output.shape}")
    obj_token = norm_decoder_output[:,:predictor.num_queries-1]
    cls_token = norm_decoder_output[:,predictor.num_queries-1:predictor.num_queries]

    print(f"obj_token: {obj_token.shape}")
    print(f"cls_token: {cls_token.shape}")

    sim = (cls_token @ obj_token.transpose(1,2)).softmax(-1)[:,0,:,None] # TODO include class token.
    cls_token = (sim * decoder_output[:,:predictor.num_queries-1]).sum(dim=1, keepdim=True)

    print(f"sim: {sim.shape}")
    print(f"cls_token: {cls_token.shape}")

    decoder_output = torch.cat((decoder_output[:,:predictor.num_queries-1], cls_token), dim=1)
    print(f"decoder_output: {decoder_output.shape}")

    class_embed = decoder_output @ predictor.class_embed
    outputs_class = predictor.lang_encoder.compute_similarity(class_embed, fake=(((not predictor.task_switch['mask']) and predictor.training)))
    print(f"class_embed {class_embed.shape}")
    print(f"outputs_class {outputs_class.shape}")

    attn_mask = torch.zeros((list(decoder_output.shape[:2]) + [size_list[0][0]*size_list[0][1]]), device=decoder_output.device).repeat(predictor.num_heads, 1, 1).bool()
    print(f"attn_mask: {attn_mask.shape}\n")
    
    results = predictor.forward_prediction_heads(output, mask_features, attn_mask_target_size=size_list[0], task='seg')

    for key, val in results.items():
        if key == "outputs_bbox" or key =="outputs_captionting":
            continue
        print(f"{key}: {val.shape}")
    
    attn_mask = results["attn_mask"]
    predictions_class.append(results["outputs_class"])
    predictions_mask.append(results["outputs_mask"])
    predictions_bbox.append(results["outputs_bbox"])
    predictions_caption.append(results["outputs_caption"])
    predictions_captioning.append(results["outputs_captionting"])

    print()
    print(predictor.num_layers, num_feature_levels)
    for i in range(predictor.num_layers):
        level_index = i % predictor.num_feature_levels
        attn_mask[torch.where(attn_mask.sum(-1) == attn_mask.shape[-1])] = False
        
        print(f"Query: {output.shape}, Key: {src[level_index].shape}")
        output, avg_attn = predictor.transformer_cross_attention_layers[i](
            output, src[level_index],
            memory_mask=attn_mask,
            memory_key_padding_mask=None,  # here we do not apply masking on padded region
            pos=pos[level_index], query_pos=query_embed
        )

        print(f"Query: {output.shape}")
        output = predictor.transformer_self_attention_layers[i](
            output, tgt_mask=self_tgt_mask,
            tgt_key_padding_mask=None,
            query_pos=query_embed
        )

        output = predictor.transformer_ffn_layers[i](
            output
        )

        print(output.shape, src[level_index].shape, attn_mask.shape)
        results = predictor.forward_prediction_heads(output, mask_features, attn_mask_target_size=size_list[(i + 1) % predictor.num_feature_levels], layer_id=i, task="seg")
        attn_mask = results["attn_mask"]
        predictions_class.append(results["outputs_class"])
        predictions_mask.append(results["outputs_mask"])
        predictions_bbox.append(results["outputs_bbox"])
        predictions_caption.append(results["outputs_caption"])
        predictions_captioning.append(results["outputs_captionting"])

    out = {
        'pred_logits': predictions_class[-1],
        'pred_masks': predictions_mask[-1],
        'pred_boxes': predictions_bbox[-1],
        'pred_captions': predictions_caption[-1],
    }
    
    for key, val in out.items():
        if key == "pred_boxes":
            continue
        print(key, val.shape)


0: torch.Size([1, 512, 25, 42]) -> torch.Size([1050, 1, 512])
0: torch.Size([1, 512, 25, 42]) -> torch.Size([1050, 1, 512])
1: torch.Size([1, 512, 50, 84]) -> torch.Size([4200, 1, 512])
1: torch.Size([1, 512, 50, 84]) -> torch.Size([4200, 1, 512])
2: torch.Size([1, 512, 100, 168]) -> torch.Size([16800, 1, 512])
2: torch.Size([1, 512, 100, 168]) -> torch.Size([16800, 1, 512])
query_pos: torch.Size([101, 1, 512])
query_feature: torch.Size([101, 1, 512])
self_tgt_mask: torch.Size([1, 178, 178])

decoder_output: torch.Size([1, 101, 512])
norm decoder_output: torch.Size([1, 101, 512])
obj_token: torch.Size([1, 100, 512])
cls_token: torch.Size([1, 1, 512])
sim: torch.Size([1, 100, 1])
cls_token: torch.Size([1, 1, 512])
decoder_output: torch.Size([1, 101, 512])
class_embed torch.Size([1, 101, 512])
outputs_class torch.Size([1, 101, 10])
attn_mask: torch.Size([8, 101, 1050])

outputs_class: torch.Size([1, 101, 10])
outputs_mask: torch.Size([1, 101, 200, 336])
attn_mask: torch.Size([8, 101, 105